## DIA 042: Integración de MLFlow para el Seguimiento de Experimentos de Retraining

En el Día 42 se añade un endpoint que permite disparar un proceso de retraining del modelo basado en feedback y, al mismo tiempo, registrar este experimento utilizando MLFlow. Con esta integración se podrá:

Registrar parámetros, métricas y artefactos del proceso de retraining.
Facilitar la comparación de resultados entre distintos experimentos.
Automatizar el seguimiento de mejoras en el modelo sin afectar el servicio de la API.
El proceso de retraining se ejecuta de forma asíncrona, y durante su ejecución se registra un experimento en MLFlow. Al finalizar, se registra una métrica (por ejemplo, la precisión) y se guarda un artefacto que simula la actualización del modelo.

Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
from datetime import datetime
from functools import wraps

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
import requests
import mlflow

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos (simplificados)
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Se asume que las credenciales son correctas
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2) y A/B Testing
# (Se incluyen para referencia; se omiten detalles ya implementados en días anteriores)
# ---------------------------
api_v1 = Flask.Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Flask.Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de Retraining Basado en Feedback (ya implementado)
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_model_endpoint():
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        time.sleep(10)  # Simula el tiempo de retraining
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Nuevo Endpoint: Pipeline de Retraining con MLFlow
# ---------------------------
@app.route('/admin/retrain_mlflow', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_mlflow():
    """
    Inicia un proceso de retraining y registra el experimento en MLFlow.
    ---
    tags:
      - Retraining
    responses:
      202:
        description: Retraining experiment logged successfully.
    """
    mlflow.set_experiment("Retraining Experiment")
    with mlflow.start_run() as run:
        logger.info("Inicio del retraining con MLFlow...")
        time.sleep(10)  # Simula el tiempo de retraining
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_metric("accuracy", 0.92)
        artifact_path = "model_info.txt"
        with open(artifact_path, "w") as f:
            f.write("Modelo actualizado basado en feedback con MLFlow.")
        mlflow.log_artifact(artifact_path)
        logger.info("Retraining completado y registrado en MLFlow.")
        return jsonify({"msg": "Retraining experiment logged in MLFlow", "run_id": run.info.run_id}), 202

# ---------------------------
# Endpoint de Reporte Automatizado (ya implementado)
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación con soporte para WebSockets (opcional) y MLFlow
# ---------------------------
if __name__ == '__main__':
    # Para desplegar con WebSocket, usamos socketio.run en lugar de app.run
    # Aquí se usa app.run para simplificar; en producción, usa socketio.run(app)
    app.run(debug=True)
Explicación del Código
Configuración y Extensiones:
Se configuran las variables de entorno, el logging, y se inicializan las extensiones necesarias (JWT, SQLAlchemy, Limiter, SocketIO, etc.).

Endpoints Comunes y Versionados:
Se incluyen endpoints básicos como /login y /health, junto con los endpoints de predicción versionados (/api/v1/predict y /api/v2/predict).

Pipeline de Retraining:
Se mantiene el endpoint /admin/retrain que inicia el retraining en un hilo separado (simulado).

Nuevo Endpoint de Retraining con MLFlow (/admin/retrain_mlflow):

Se configura MLFlow para registrar el experimento en el experimento "Retraining Experiment".
Durante el proceso de retraining (simulado con un sleep de 10 segundos), se registran un parámetro (learning_rate) y una métrica (accuracy).
Se genera y registra un artefacto (archivo "model_info.txt") que simula la actualización del modelo.
Al finalizar, se devuelve el ID del run de MLFlow.
Endpoint de Reporte Automatizado:
Se genera un reporte resumido basado en los datos de feedback.

Ejecución de la Aplicación:
La aplicación se ejecuta en modo debug. En producción, se recomienda utilizar un servidor adecuado (por ejemplo, con soporte para WebSocket mediante socketio.run(app)).

